### 配置文件验证

In [ ]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('config.yaml') # 读取配置文件

In [ ]:
from modelModule.model4 import VAE4
import torch

In [ ]:
model = VAE4(dim=57, nhead=3)
a = torch.randn((10,57))
b = torch.randn((10,57))
x, y, z = model(a,b)

In [ ]:
x

### dataloader验证

In [11]:
import torch
from dataModule.dataset1 import FlatDataset
from torch.utils.data import DataLoader

In [12]:
dataset = FlatDataset('Credit_Card无标签.csv')
print(dataset.Max_Val.shape)
print(dataset.Min_Val)

(23,)
[ 1.00000e+04  1.00000e+00  0.00000e+00  0.00000e+00  2.10000e+01
 -2.00000e+00 -2.00000e+00 -2.00000e+00 -2.00000e+00 -2.00000e+00
 -2.00000e+00 -1.65580e+05 -6.97770e+04 -1.57264e+05 -1.70000e+05
 -8.13340e+04 -3.39603e+05  0.00000e+00  0.00000e+00  0.00000e+00
  0.00000e+00  0.00000e+00  0.00000e+00]


In [ ]:
dataset = FlatDataset('Letter_train.csv')
train_dataloader = DataLoader(dataset, 
                             batch_size=10, 
                             collate_fn=dataset.collater, 
                             shuffle=True,
                             num_workers=2)
batch = None
for i in train_dataloader:
    batch = i
    break

In [ ]:
test_data = imputed_data * global_max + global_min # 广播效应

In [ ]:
a = torch.randn(2,3,4)
b = torch.nn.AdaptiveMaxPool1d(output_size=1)(a).squeeze(-1)

In [ ]:
b.shape

### 模型精度测定

In [2]:
from dataModule.dataset1 import FlatDataset
# from modelModule.model1 import VAE
# from modelModule.model2 import VAE2
from modelModule.model4 import VAE4
from torch.utils.data import DataLoader
from utils import result_show, get_missing
import torch
import numpy as np

In [3]:
## 模型参数载入
model = VAE4(dim=23) # nhead=3
checkpoint_path = r'模型参数保存/Credit_model4_norm_way_mean_norm_/version_1/checkpoints/epoch=38-step=936.ckpt'    
data_norm = 'mean_norm'  # 数据集采用正则化方式minmax_norm,mean_norm
train_csv = r'Credit_Card_test.csv'
test_csv = r'Credit_Card_test.csv'

checkpoint = torch.load(checkpoint_path)
model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
model.eval()
## 载入训练集的均值(最小)方差(最大)
train_dataset = FlatDataset(train_csv, data_norm=data_norm)
model.get_global_min_max(global_max=train_dataset.Max_Val, global_min=train_dataset.Min_Val)
print('载入成功')

载入成功


In [4]:
test_dataset = FlatDataset(test_csv, data_norm=data_norm)
data_size = 10
src_data = np.stack([test_dataset[index]['src_data'] for index in range(data_size)], axis=0)
normal_data = np.stack([test_dataset[index]['global_normal_data'] for index in range(data_size)], axis=0)
_, miss_matrix = get_missing(src_data, 0.3) # 得到缺失矩阵
input = src_data.copy()
input[miss_matrix == 0] = np.nan

In [5]:
imputed_data, output = model.inference(input, miss_matrix) # inference输出的结果已经重构后的结果
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()

In [6]:
for i in range(23):
    print(src_data[0][i], '————', imputed_data[0][i])

20000.0 ———— 178112.7023406656
1.0 ———— 2.048616142763939
1.0 ———— 2.4944369647200086
2.0 ———— 1.6519161200004289
33.0 ———— 41.84181396285687
1.0 ———— -0.855229797030175
2.0 ———— 0.5070426545188023
2.0 ———— 0.7002046422030956
2.0 ———— -0.622437304385453
2.0 ———— -0.37793545004572837
2.0 ———— 0.010196601081779999
17971.0 ———— 43187.45982283997
17399.0 ———— 75306.10562190472
19057.0 ———— 11523.675331314771
18453.0 ———— 99941.03170029158
19755.0 ———— 16370.257948483919
19288.0 ———— 50622.99398412446
0.0 ———— 13858.377374257103
2260.0 ———— -4716.746038206365
0.0 ———— 15957.81672385339
1600.0 ———— -18219.822489066777
0.0 ———— -4980.968232046074
644.0 ———— -10223.906387798277


In [7]:
imputed_data[2]

array([ 1.78060508e+05,  2.04800016e+00,  2.49457248e+00,  1.65175908e+00,
        4.18314702e+01, -8.54966291e-01,  5.05183088e-01,  6.99807794e-01,
       -6.22939560e-01, -3.77302437e-01,  1.01128512e-02,  4.31802048e+04,
        7.53119946e+04,  1.15119701e+04,  9.99450713e+04,  1.63882150e+04,
        5.06505704e+04,  1.38719633e+04, -4.71215145e+03,  1.59658535e+04,
       -1.82256102e+04, -4.97079467e+03, -1.02158485e+04])

In [8]:
imputed_data[1]

array([ 1.78138418e+05,  2.04859699e+00,  2.49391543e+00,  1.65196771e+00,
        4.18412027e+01, -8.54825750e-01,  5.07254279e-01,  6.99441124e-01,
       -6.21976413e-01, -3.77784740e-01,  9.84456958e-03,  4.32007779e+04,
        7.52916793e+04,  1.15771843e+04,  9.98913009e+04,  1.63923609e+04,
        5.05762176e+04,  1.38454550e+04, -4.71526506e+03,  1.59492273e+04,
       -1.82043965e+04, -4.97782529e+03, -1.02178514e+04])

In [ ]:
result_show(src_data, imputed_data, miss_matrix)

In [ ]:
imputed_data

In [ ]:
imputed_data.shape

In [ ]:
# test_dataset = FlatDataset('Letter_test.csv', data_norm='mean_norm') # minmax_norm
# index = 102
# src_data, M_matrix, miss_data = test_dataset[index]['src_data'].reshape(1,-1), test_dataset[index]['global_miss_matrix'].reshape(1,-1), test_dataset[index]['global_miss_data'].reshape(1,-1)
# src_data = test_dataset[index]['src_data'].reshape(1,-1)
# normal_data = test_dataset[index]['global_normal_data'].reshape(1,-1)
# miss_data, M_matrix = get_missing(normal_data, 0.3) # 产生掩码数据，并采用随机数填补

# imputed_data, _, _ = model(torch.from_numpy(miss_data).float(), torch.from_numpy(M_matrix).float()) 
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy() # 得到最终插补结果,数据由[0,1]范围内
# res_imputed_data = np.around(restore_data(imputed_data, test_dataset.Max_Val, test_dataset.Min_Val))
# result_show(src_data, res_imputed_data, M_matrix)

In [ ]:
# batch测试
# src_data, miss_data, M_matrix = batch['src_data'], batch['miss_data'], batch['miss_matrix']
# normal_data = batch['normal_data']
# imputed_data, _, _ = model(miss_data, M_matrix)
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()
# res_imputed_data = np.around(restore_data(imputed_data, dataset.Max_Val, dataset.Min_Val))